In [17]:
# --quiet parametresini kaldırdık, böylece ne olduğunu görebileceğiz
# Temel LangChain ve yardımcı kütüphaneler (bunlar aynı kalıyor)
!pip3 install --upgrade --quiet langchain langchain-community chromadb pypdf pandas streamlit python-dotenv

# Google kütüphanesi yerine OLLAMA için GEREKLİ kütüphane
!pip3 install --upgrade --quiet ollama


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# --- LangChain Modülleri ---

# 1. Döküman Yükleyiciler ve Bölücüler (Bunlar aynı kalıyor)
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_community.vectorstores import Chroma
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings # <- YEREL EMBEDDING İÇİN

# 4. LangChain Çalıştırma ve Şablon Modülleri (Aynı kalıyor)
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field # <- Deprecation uyarısını gidermek için güncellendi
import os
import tempfile
import streamlit as st
import pandas as pd
# from dotenv import load_dotenv <- ARTIK API ANAHTARI KULLANMADIĞIMIZ İÇİN GEREKSİZ BİR KOD PARÇASI BİZİM İÇİN

In [22]:
llm = ChatOllama(model="deepseek-r1")
llm.invoke("Tell me a joke about cats")
# 'llm' nesnesini kullanarak yerel modele isteğimizi gönderiyoruz.
response = llm.invoke("Nasılsın kralım")
# Gelen cevabın içeriğini yazdırıyoruz.
print(response.content)

<think>
Hmm, kullanıcı “Nasılsın kralım” der. Bu esmera土耳其tlyi bir ünvan sormuş. Belki de oyun veya bir虚构角色 hakkında merak ediyor olabilir.

Bu ifadeyi iki parçaya ayıralım: “nasılsın” ve “kralım”. “Nasılsın” genel bir soru, bu yüzden kullanıcı belki sohbet başlatmak istemiş. “Kralım” ise belki oyun dünyasından bir şey veya takma bir isim olabilir.

Şimdi, kullanıcının Türkçe konuştuğunu varsayıyoruz çünkü “kralım” kelimesi Türkiye'de yaygın olarak kullanılıyor (“kral” = imparator). Belki de kullanıcı esmera bir oyundan veya karakterden bahsediyor. Eğer “esmerapiun” gibi bir şeyseydi…… ama bu kesin değil, sadece tahminim.

Bu yüzden cevabımda iki yönlü yaklaşım kullanmalıyım: hem genel sohbet modunda olmalı hem de belki esmera kelimesini anlama adımı atlamalıyım. Eğer kullanıcı daha spesifik bir konu hakkında bilgi isterse, onu teşviq etmeliyim.

Kullanıcının yaşı veya cinsiyeti hakkında bir ipucu yok ama “kralım” kelimesi genelde samimi ve dostça anlaşıldığı için olasılıklı daha genç 

In [24]:

loader = PyPDFLoader("C:/Users/hamdi/Desktop/rag_llms/data/174-182 Large Language Models vs Human Interpretation Which is More Accurate in Text Classification.pdf")
pages = loader.load()
pages

[Document(metadata={'producer': 'Microsoft® Word 2016', 'creator': 'Microsoft® Word 2016', 'creationdate': '2025-07-08T15:09:49+03:00', 'author': '-', 'moddate': '2025-07-08T15:29:31+03:00', 'subject': 'IEEE Transactions on Magnetics', 'title': '\uf020', 'source': 'C:/Users/hamdi/Desktop/rag_llms/data/174-182 Large Language Models vs Human Interpretation Which is More Accurate in Text Classification.pdf', 'total_pages': 9, 'page': 0, 'page_label': '1'}, page_content='BALKAN JOURNAL OF ELECTRICAL & COMPUTER ENGINEERING,  Vo l. 13, No. 2, June 2025  \nCopyright © BAJECE  ISSN: 2147-284X  http://dergipark.gov.tr/bajece \nResearch Article \nAbstract— Ekşi Sözlük is a widely used social network where \nnumerous unusual events are discussed. In this context, it serves \nas a real -time news source for emergency response teams and \ndigital news platforms. In this study, a dataset was compiled from \ncomments shared on the Ekşi Sözlük platform regarding the \nKahramanmaraş eart hquake on Febr

In [25]:

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500,
                                            chunk_overlap=200,
                                            length_function=len,
                                            separators=["\n\n", "\n", " "])
chunks = text_splitter.split_documents(pages)

In [31]:

def get_embedding_function():
    """
    Yerel Ollama servisinden bir embedding fonksiyonu oluşturur ve döndürür.
    """
    embeddings = OllamaEmbeddings(model="nomic-embed-text")
    return embeddings

embedding_function = get_embedding_function()
test_vector = embedding_function.embed_query("cat")

In [32]:

evaluator = load_evaluator(evaluator="embedding_distance", 
                            embeddings=embedding_function)

evaluator.evaluate_strings(prediction="Amsterdam", reference="coffeeshop")

{'score': 0.3489810034414459}

In [33]:

evaluator.evaluate_strings(prediction="Paris", reference="London")

{'score': 0.18615306960721245}

In [ ]:
# Benzersiz kimlik (ID) oluşturmak için gerekli kütüphanemiz.
import uuid
# Database verktör için import ettim.
from langchain_community.vectorstores import Chroma

def create_vectorstore(chunks, embedding_function, vectorstore_path):
    """
    Bu fonksiyon, metin parçalarını (chunks) alır, bunları vektörlere dönüştürür 
    ve kalıcı bir Chroma veritabanına kaydeder.
    """

    # 1. ADIM: Her bir metin parçası için benzersiz bir ID oluşturma
    # doc.page_content (metnin kendisi) kullanılarak her bir metin için tekrarlanabilir
    # (aynı metne her zaman aynı ID'yi veren) ve benzersiz bir kimlik (ID) oluşturulur.
    # Bu sayede, aynı metin parçasının veritabanına tekrar eklenmesi önlenir.
    ids = [str(uuid.uuid5(uuid.NAMESPACE_DNS, doc.page_content)) for doc in chunks]
    
    # 2. ADIM: Kopya olan metinleri ayıklama
    # Daha önce eklenmiş ID'leri hızlıca kontrol etmek için boş bir küme (set) oluşturulur.
    unique_ids = set()
    # Kopyası olmayan, benzersiz metin parçalarını tutacak boş bir liste oluşturulur.
    unique_chunks = [] 
    
    # Her bir 'chunk' ve ona karşılık gelen 'id' üzerinde bir döngü başlatılır.
    for chunk, id in zip(chunks, ids):     
        # Eğer bu 'id' daha önce 'unique_ids' kümesine eklenmemişse (yani bu bir kopya değilse)...
        if id not in unique_ids:       
            # ...bu yeni 'id'yi kümeye ekle, böylece bir dahaki sefere kopya olduğu anlaşılır.
            unique_ids.add(id)
            # ...ve bu benzersiz 'chunk'ı listeye ekle.
            unique_chunks.append(chunk) 

    # 3. ADIM: Vektör Veritabanını Oluşturma
    # Ayıklanan benzersiz metin parçalarını ('unique_chunks') kullanarak bir Chroma veritabanı nesnesi oluşturur.
    vectorstore = Chroma.from_documents(documents=unique_chunks, 
                                        # Her bir metne karşılık gelen benzersiz ID'ler listesi.
                                        ids=list(unique_ids),
                                        # Metinleri sayısallaştıracak (vektöre çevirecek) olan fonksiyon.
                                        embedding=embedding_function, 
                                        # Oluşturulan veritabanının diske (bilgisayara) kaydedileceği klasör yolu.
                                        persist_directory = vectorstore_path)

    # 4. ADIM: Veritabanını Kalıcı Olarak Kaydetme
    # Bellekte oluşturulan veritabanını, yukarıda belirtilen 'persist_directory' yoluna kalıcı olarak kaydeder.
    # Bu sayede programı kapatıp açtığınızda veritabanını yeniden oluşturmak zorunda kalmazsınız.
    vectorstore.persist()
    
    # 5. ADIM: Oluşturulan Nesneyi Döndürme
    # Oluşturulan ve kullanıma hazır veritabanı nesnesini fonksiyonun çıktısı olarak döndürür.
    return vectorstore

In [37]:
# Create vectorstore
vectorstore = create_vectorstore(chunks=chunks, 
                                 embedding_function=embedding_function, 
                                 vectorstore_path="vectorstore_test")

C:\Users\hamdi\AppData\Local\Temp\ipykernel_43448\1894989826.py:46: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [38]:
retriever = vectorstore.as_retriever(search_type="similarity")
relevant_chunks = retriever.invoke("What is the title of the paper?")
relevant_chunks

[Document(metadata={'producer': 'Microsoft® Word 2016', 'page': 3, 'creationdate': '2025-07-08T15:09:49+03:00', 'subject': 'IEEE Transactions on Magnetics', 'title': '\uf020', 'creator': 'Microsoft® Word 2016', 'source': 'C:/Users/hamdi/Desktop/rag_llms/data/174-182 Large Language Models vs Human Interpretation Which is More Accurate in Text Classification.pdf', 'moddate': '2025-07-08T15:29:31+03:00', 'page_label': '4', 'total_pages': 9, 'author': '-'}, page_content='BALKAN JOURNAL OF ELECTRICAL & COMPUTER ENGINEERING,     Vol. 13, No. 2, June 2025                                              \n                                              \n \nCopyright © BAJECE                                                                ISSN: 2147-284X                                                     http://dergipark.gov.tr/bajece        \nclassification model and enhancing the efficiency of the manual \nlabeling process. Ensuring an equal number of examples from \neach category (250 labeled as

In [39]:

# Prompt template
PROMPT_TEMPLATE = """
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer
the question. If you don't know the answer, say that you
don't know. DON'T MAKE UP ANYTHING.

{context}

---

Answer the question based on the above context: {question}
"""

In [40]:

# Concatenate context text
context_text = "\n\n---\n\n".join([doc.page_content for doc in relevant_chunks])

# Create prompt
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, 
                                question="What is the title of the paper?")
print(prompt)

Human: 
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer
the question. If you don't know the answer, say that you
don't know. DON'T MAKE UP ANYTHING.

BALKAN JOURNAL OF ELECTRICAL & COMPUTER ENGINEERING,     Vol. 13, No. 2, June 2025                                              
                                              
 
Copyright © BAJECE                                                                ISSN: 2147-284X                                                     http://dergipark.gov.tr/bajece        
classification model and enhancing the efficiency of the manual 
labeling process. Ensuring an equal number of examples from 
each category (250 labeled as ‘0’ and 250 as ‘1’) was essential 
for enabling the model to perform consistently across both 
classes and for ensuring more reliable human intervention 
during the evaluation phase. 
 
 
Fig. 1. Flow Diagram of the Study 
V. MODEL EXPLANATION 
During the classificati

In [41]:
llm.invoke(prompt)

AIMessage(content='<think>\nOkay, so I need to answer the user\'s query about the title of the paper based on the provided context. Let me start by carefully examining the given text.\n\nThe first thing that comes to mind is looking for a clear header or footer where titles are usually placed. Scanning through each segment, I see references and tables but no explicit title mentioned here. The user might be confused if they expected the title to be in this context, so maybe they think there\'s an error because it wasn\'t found.\n\nWait, let me check again. There\'s a mention of "BALKAN JOURNAL OF ELECTRICAL & COMPUTER ENGINEERING" and some table titles like Table III and Table VII, but nothing about the paper\'s title itself. The references also don\'t include the title. Hmm, maybe the user is referring to another part of the document not included here.\n\nBut according to the instructions, only the provided context should be used. Since there\'s no title in the given text, it makes sen

In [42]:

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
            {"context": retriever | format_docs, "question": RunnablePassthrough()}
            | prompt_template
            | llm
        )
rag_chain.invoke("What's the title of this paper?")

AIMessage(content='<think>\nAlright, let me try to figure out what the title of this paper is. So, looking at the provided context, I see that it starts with "BALKAN JOURNAL OF ELECTRICAL & COMPUTER ENGINEERING" and mentions Table VII about classification metrics. There are also sections like V. MODEL EXPLANATION and references ending with [18]. Wait, but none of these seem to mention a specific title for the paper itself.\n\nHmm, maybe the title is in the reference part? Let me check each citation. The first one seems to be related to crisis informatics. Then there\'s a Turkish study about tweets using Google BERT model. There are mentions of attention mechanisms and mental health analysis on social media. But these references might not directly give the paper\'s title.\n\nAlternatively, perhaps the paper is part of a special issue or volume in that journal. The citation says Vol. 13, No. 2, June 2025. Without more information like author names or specific sections outside this contex

In [45]:
class AnswerWithSources(BaseModel):
    """An answer to the question, with sources and reasoning."""
    answer: str = Field(description="Answer to question")
    sources: str = Field(description="Full direct text chunk from the context used to answer the question")
    reasoning: str = Field(description="Explain the reasoning of the answer based on the sources")
    
class ExtractedInfo(BaseModel):
    """Extracted information about the research article"""
    paper_title: AnswerWithSources
    paper_summary: AnswerWithSources
    publication_year: AnswerWithSources
    paper_authors: AnswerWithSources

In [47]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# --- BU MODELLER ZATEN TANIMLIYDI ---
# class AnswerWithSources(BaseModel):
#     ...
# class ExtractedInfo(BaseModel):
#     ...
# --------------------------------------

# 1. ADIM: Parser'ı ve Prompt Şablonunu Hazırla

# JSON çıktısını, senin ExtractedInfo modeline dönüştürecek olan aracı oluştur.
parser = JsonOutputParser(pydantic_object=ExtractedInfo)

# LLM'e JSON formatında nasıl cevap vereceğini anlatan yeni bir prompt şablonu oluştur.
JSON_PROMPT_TEMPLATE = """
You are an expert extraction agent. From the given context, extract the requested information.
If a piece of information is not available in the context, use an empty string "" for that field.
Your output must be a valid JSON object.

Here are the precise formatting instructions:
{format_instructions}

CONTEXT:
{context}

---

Based on the context, extract the information for the user's query:
QUERY: {question}
"""

# Şablonu, parser'dan gelen format talimatlarıyla birleştir.
prompt_template = ChatPromptTemplate.from_template(
    template=JSON_PROMPT_TEMPLATE,
    partial_variables={"format_instructions": parser.get_format_instructions()}
)


# 2. ADIM: Zinciri Oluştur ve Çalıştır (Sorduğun Kısım)

# Artık tüm parçalar hazır olduğuna göre, montaj hattını (zinciri) kur.
rag_chain = (
            {"context": retriever | format_docs, "question": RunnablePassthrough()}
            | prompt_template
            | llm
            | parser  # LLM'den gelen metni alıp ExtractedInfo nesnesine çevirecek.
        )

# Zinciri çalıştır ve sonucu al.
result = rag_chain.invoke("Give me the title, summary, publication date, and authors of the research paper.")

# Sonucu ekrana yazdır.
print(result)

{'paper_title': {'answer': '', 'sources': '', 'reasoning': ''}, 'paper_summary': {'answer': 'This research study aimed to automatically classify comments on Ekşi Sözlük related to the 6th February 2023 Kahramanmaraş earthquake into four categories (Source Information, Emotional Response, Social Inference, Personal Experience) using a Gemma2 model. The analyses focused on evaluating classification performance across different categories based on metrics derived from confusion matrices.', 'sources': '', 'reasoning': ''}, 'publication_year': {'answer': '2025', 'sources': 'BALKAN JOURNAL OF ELECTRICAL & COMPUTER ENGINEERING, Vol. 13, No. 2, June 2025', 'reasoning': "The publication year is explicitly mentioned in the copyright line and table header as 'June 2025'."}, 'paper_authors': {'answer': '["Ahmet Hamdi Özkurt", "Emrah Aydemir", "Yasin Sönmez"]', 'sources': "\n\nBIOGRAPHIES \nAhmet Hamdi Özkurt is going on his Bachelor's degree in Management Information System at Sakarya University. 

In [51]:
structured_response = rag_chain.invoke("Give me the title, summary, publication date, authors of the research paper.")
df = pd.DataFrame([structured_response])

# Transforming into a table with two rows: 'answer' and 'source'
answer_row = []
source_row = []
reasoning_row = []

for col in df.columns:
    answer_row.append(df[col][0]['answer'])
    source_row.append(df[col][0]['sources'])
    reasoning_row.append(df[col][0]['reasoning'])

# Create new dataframe with two rows: 'answer' and 'source'
structured_response_df = pd.DataFrame([answer_row, source_row, reasoning_row], columns=df.columns, index=['answer', 'source', 'reasoning'])
structured_response_df

,paper_title,paper_summary,publication_year,paper_authors
answer,Automatic Classification of Social Media Comme...,This research aims to automatically classify c...,,
source,The paper title is mentioned at the end as par...,This summary is derived from multiple parts of...,,No author names are explicitly mentioned in th...
reasoning,The context ends with information about the pu...,Extracted key elements from the text to form a...,,The context does not list any authors or conta...
